## <font color=yellow>1. Inference with orch</font>
- orch는 agent와 키워드를 추출해주는 모듈

In [1]:
from azure.gpt.orch import Orch

orch = Orch()

history = """
user : 한국 수도는 어디
"""
response = orch._generate(history)
print(response)

{"agent": "Web agent", "keyword": "한국 수도"}


In [2]:
history = """
user : 한국 수도는 어디
assistant : 한국 수도는 서울입니다.
user : 그럼 그거 관련한 뉴스 알려줄래?
"""
response = orch._generate(history)
print(response)

{"agent": "News agent", "keyword": "서울 관련 뉴스"}


한국 수도는 어디냐는 질문에, 
- web으로 한국 수도라는 키워드로 찾을 수 있도록 잘 구현이 됨

관련 뉴스가 뭐냐는 질문에,
- 뉴스 agent와 키워드가 잘 선택이 됨

## <font color=yellow>2. AI Search with MS Azure</font>

## <font color=red>[test] embedding model</font>

In [1]:
import os
import re
import pandas as pd 

df = pd.read_csv(os.path.join(os.getcwd(),'bill_sum_data.csv')) 
df.head()

,Unnamed: 0,bill_id,text,summary,title,text_len,sum_len
0,0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278


In [2]:
df_bills = df[['text', 'summary', 'title']]
df_bills.head()

,text,summary,title
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...


In [3]:
pd.options.mode.chained_assignment = None 

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))
df_bills.head()

,text,summary,title
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...


In [4]:
from azure import generate_embeddings

df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))
df_bills.head()

,text,summary,title,ada_v2
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,"[-0.02267872914671898, -0.007992359809577465, ..."
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,"[-0.04170488566160202, -0.009005059488117695, ..."
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,"[-0.04132331907749176, -0.003348014783114195, ..."
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,"[-0.01324405800551176, -0.007176324725151062, ..."
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,"[-0.039218343794345856, -0.025535382330417633,..."


## <font color=red>Data를 Azure DB에 저장</font>

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


file_name = "냉장고_메뉴얼2.pdf"
file_path = fr'.\azure\cognitive_search\{file_name}'

loader = PyPDFLoader(file_path)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len
)
pages = loader.load_and_split(text_splitter)
len(pages)

138

In [6]:
pages[0]

Document(metadata={'producer': 'Acrobat Distiller 23.0 (Windows)', 'creator': 'FrameMaker 2017.0.4', 'creationdate': '2023-08-07T08:48:21+00:00', 'author': 'soyoung94.jang', 'moddate': '2023-08-07T08:52:34+09:00', 'title': 'ko-kr_main.book.book', 'source': '.\\azure\\cognitive_search\\냉장고_메뉴얼2.pdf', 'total_pages': 36, 'page': 0, 'page_label': '1'}, page_content="ࢲݺࢸ\nպ\u0a62Ҋ\nਃ\x0f\nਃ\x0f\nਃ\x0f\n֙\n\x0f\u05eeפ\n\n\x0f\u05eeפ\n\x16\x14\x19\x11\x13\x18\x11\x18-'.\n3FW\x0f\x11\x18@\x11\x19\x11\x18\x13\x14")

pdf로부터 텍스트를 추출했다고 가정
- 아래는 의미없는 것들을 걸러내기 위한 함수임
- ex 특수문자 같은 것들

In [7]:
import re

def is_korean_english_ratio_high(text, threshold=0.8):
    # 한글과 영어 문자를 찾기 위한 정규 표현식
    korean_pattern = re.compile(r'[가-힣]')
    english_pattern = re.compile(r'[a-zA-Z]')

    # 한글과 영어 문자의 개수 세기
    korean_count = len(korean_pattern.findall(text))
    english_count = len(english_pattern.findall(text))

    # 전체 문자에서 한글과 영어 문자의 비율 계산
    total_count = len(text.replace(' ', ''))
    if total_count == 0:
        return False  # 텍스트가 비어있으면 False 반환

    ratio = (korean_count + english_count) / total_count
    return ratio >= threshold

In [1]:
from azure.cognitive_search.ai_search import AISearch

ai_search = AISearch()

In [9]:
contexts = []

# 정확히는 page는 아님
for page in pages:
    if is_korean_english_ratio_high(page.page_content):
        contexts.append({
            'context' : page.page_content,
            'file_name': file_name
        })
ai_search.upload(contexts)

Upload Success


<Response [200]>

## <font color=red>Data Search</font>

In [3]:
file_name = "냉장고_메뉴얼2.pdf"

search_documents = ai_search.search(
    query='noise',
    file_name=file_name,
    topk=3
)

print(search_documents)

Search Success
[{'@search.score': 0.01666666753590107, 'id': '029dd98adde7dad14726be13efa3561f990ad488', 'context': '소음이나 이상한 소리가 \n나요.\n냉장고를 바닥이 약하거나 평평하지 않은 곳에 설치했나요?\n• 냉장고는 바닥이 단단하고 수평인 곳에 설치하세요.\n냉장고 뒷면이 벽에 닿았나요?\n• 냉장고를 천장과 좌우 벽면에서 최소 10 cm 이상 떨어진 곳에 설치하세요.\n냉장고 뒤나 위에 물건이 떨어져 있나요?\n• 냉장고 뒤 또는 위에 있는 물건을 치우세요.\n냉장고에서 소리가 \n들려요.\n“딱” “뚝” 하는 소리가 들리나요?\n• 냉장고 안의 온도 변화에 따라 여러 부품들이 수축 또는 팽창하면서 나는'}, {'@search.score': 0.016393441706895828, 'id': '9d980d18ff06fd6d577f9dd21e57b7c4fd0fe141', 'context': '“꾸르륵~” 하며 물 흐르는 소리가 들리나요?\n• 냉장실이나 냉동실에서 냉매의 상태가 액체 또는 기체로 바뀌면서 소리가 \n납니다.\n문을 닫고 바로 “슈~” “ 쉭~” 하는 바람 소리가 들리나요?\n• 냉장고로 들어온 더운 공기가 빨리 차가워지면, 냉장고 안에서 공기를 차갑게 \n만드는 냉매의 상태가 바뀌면서 소리가 납니다.\n냉장고에서 떨리는 소리가 들리나요?\n• 마루나 평평하지 않은 바닥 위, 나무 소재로 된 벽 앞에 냉장고를 설치하면 진동 \n때문에 냉장고의 떨림 소리가 커질 수 있습니다.\n증상 원인 및 해결책'}, {'@search.score': 0.016129031777381897, 'id': '47610f320b70c2a7134c8f9f454d3e2adf30491b', 'context': '냉장고에서 소리가 \n들려요.\n“딱” “뚝” 하는 소리가 들리나요?\n• 냉장고 안의 온도 변화에 따라 여러 부품들이 수축 또는 팽창하면서 나는 \n소리입니다. 냉장고의

In [4]:
print(search_documents[0]['context'])

소음이나 이상한 소리가 
나요.
냉장고를 바닥이 약하거나 평평하지 않은 곳에 설치했나요?
• 냉장고는 바닥이 단단하고 수평인 곳에 설치하세요.
냉장고 뒷면이 벽에 닿았나요?
• 냉장고를 천장과 좌우 벽면에서 최소 10 cm 이상 떨어진 곳에 설치하세요.
냉장고 뒤나 위에 물건이 떨어져 있나요?
• 냉장고 뒤 또는 위에 있는 물건을 치우세요.
냉장고에서 소리가 
들려요.
“딱” “뚝” 하는 소리가 들리나요?
• 냉장고 안의 온도 변화에 따라 여러 부품들이 수축 또는 팽창하면서 나는


## <font color=yellow>3. Web Search</font>

In [1]:
from azure.google_search.web import Web

web = Web()
web.search("삼성전자")

[{'title': 'Samsung 대한민국 | 모바일 | TV | 가전 | IT',
  'snippet': '삼성전자 및 삼성전자 임직원은 당사가 공식적으로 요청하지 않았음에도 불구하고 여러분께서 일방적으로 당사에 제출하는 아이디어나 제안 등을 수령하거나 검토하지\xa0...',
  'url': 'https://www.samsung.com/sec/'},
 {'title': '삼성전자 뉴스룸',
  'snippet': "최신기사 · 삼성전자, '2025 대한민국 올해의 녹색상품'서 대거 선정 · 더위를 뚫은 열기… · 삼성전자, 갤럭시 워치 울트라 기존 모델에 최신 '원 UI 8 워치' 업데이트\xa0...",
  'url': 'https://news.samsung.com/kr/'},
 {'title': '삼성전자(005930)',
  'snippet': 'no content',
  'url': 'https://finance.naver.com/item/main.nhn?code=005930'}]

In [1]:
from azure.google_search.news import News

news = News()
news.search("트럼프 대통령")

[{'title': '홈페이지 - BBC News 코리아',
  'snippet': '6 hours ago ... 트럼프 대통령과 이시바 총리 · 교복 차림의 파르한 하산이 어른들에 둘러싸여 카메라를 응시하는 모습 · A woman weeps in the hospital · 남매가 포옹하고 있는 모습\xa0...',
  'url': 'https://www.bbc.com/korean'},
 {'title': "트럼프, 일본과 '대규모' 무역 협상 타결...상호 관세 15% - BBC News ...",
  'snippet': '4 hours ago ... 도널드 트럼프 미국 대통령은 22일 일본과 무역 협상이 완료되었다면서 일본은 15%의 상호 관세를 낼 것이라고 주장했다.',
  'url': 'https://www.bbc.com/korean/articles/cm20lky2kvyo'},
 {'title': '이재명 정부 1기 내각 인사청문회, 누가 통과했으며 누가 임명됐나 ...',
  'snippet': '2 days ago ... 21일 기준 6명의 신임 장관이 공식 임명된 가운데 이재명 대통령이 이진숙 교육부 장관 후보자에 대한 지명을 철회했다. 이재명 정부 내각 가운데 첫\xa0...',
  'url': 'https://www.bbc.com/korean/articles/c74zp57gpnvo'}]

In [2]:
news.search("삼성전자")

[{'title': '홈페이지 - BBC News 코리아',
  'snippet': "6 hours ago ... 이재용 삼성전자 회장. 사법 리스크 털어낸 삼성 이재용, '위기의 삼성' 구할까? 2025년 7월 17일. 드루즈족의 깃발을 들고 있는 사람들. 이스라엘은 왜 시리아를 공격\xa0...",
  'url': 'https://www.bbc.com/korean'},
 {'title': "삼성 이재용, '10년 사법 리스크' 벗었다…앞으로의 과제는? - BBC ...",
  'snippet': "6 days ago ... 이재용 삼성전자 회장이 '불법 승계' 관련 무죄 판결 확정으로 모든 사법 리스크를 털어냈다. 17일 대법원 3부(주심 오석준 대법관)는 부당합병·회계\xa0...",
  'url': 'https://www.bbc.com/korean/articles/cql01wdv7e3o'},
 {'title': '윤석열, Page 9 / 10 - BBC News 코리아',
  'snippet': 'Mar 29, 2025 ... 미국 대통령이 첫 일정으로 삼성전자 공장을 찾은 이유. 2022년 5월 21일. 윤석열 대통령과 조 바이든 미국 대통령이 20일 경기 평택 삼성전자 반도체. 오늘 한미정상회담\xa0...',
  'url': 'https://www.bbc.com/korean/topics/cne1xwzg9zpt?page=9'}]